In [1]:
## load packages
## if you haven't installed those packages yet (for example, if you don't have pandas), please type in your terminal: pip install pandas
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
import csv

In [4]:
# please put this script and 'List of companies.xlsx'(your file) in the same directory
directory = os.getcwd() 
file_name = 'List of companies.xlsx'
path = os.path.join(directory,file_name)
df = pd.read_excel(path,header= 0)
cik = [df['cik'][i] for i in range(len(df))] # get all the cik

In [16]:
def get_url1(obj):
    table = obj.find_all('table',class_ = 'tableFile2')
    if table[0].a==None:
        return False
    else:
        return table[0].a['href']

In [17]:
def get_url2(obj):
    table = obj.find_all('table',class_ = 'tableFile')
    if table[0].a==None:
        return False
    else:
        return table[0].a['href']

In [34]:
url_first = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK='
url_last = '&type=def+14&dateb=&owner=exclude&count=40'
url_prefix = 'https://www.sec.gov'

cik_archives = []
web1_archives = []
web2_archives = []

for i in range(len(df)):
    url = url_first+(str(cik[i]))+url_last
    data = requests.get(url).content
    soup = BeautifulSoup(data, 'html.parser')
    web = get_url1(soup)
    if web == False:
        continue
    url2 = url_prefix + web.encode('utf-8')
    data2 = requests.get(url2).content
    soup2 = BeautifulSoup(data2,'html.parser')
    web2 = get_url2(soup2)
    if web2 == False:
        continue
    url3 = url_prefix + web.encode('utf-8')
    cik_archives.append(cik[i])
    web1_archives.append(url2) 
    web2_archives.append(url3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [45]:
## build a pandas format
web_url = pd.DataFrame(
    {'cik': cik_archives,
    'web1': web1_archives,
    'web2': web2_archives
    })

In [48]:
## write the data to a csv file
## note this file is in the same directory of this script
web_url.to_csv('web_url.csv',index = False)

In [49]:
## look and check
web_url

,cik,web1,web2
0,1394954,https://www.sec.gov/Archives/edgar/data/139495...,https://www.sec.gov/Archives/edgar/data/139495...
1,806085,https://www.sec.gov/Archives/edgar/data/806085...,https://www.sec.gov/Archives/edgar/data/806085...
2,1401106,https://www.sec.gov/Archives/edgar/data/140110...,https://www.sec.gov/Archives/edgar/data/140110...
3,1499912,https://www.sec.gov/Archives/edgar/data/149991...,https://www.sec.gov/Archives/edgar/data/149991...
4,356708,https://www.sec.gov/Archives/edgar/data/356708...,https://www.sec.gov/Archives/edgar/data/356708...
5,43300,https://www.sec.gov/Archives/edgar/data/43300/...,https://www.sec.gov/Archives/edgar/data/43300/...
6,1023024,https://www.sec.gov/Archives/edgar/data/102302...,https://www.sec.gov/Archives/edgar/data/102302...
7,1002590,https://www.sec.gov/Archives/edgar/data/100259...,https://www.sec.gov/Archives/edgar/data/100259...
8,1093434,https://www.sec.gov/Archives/edgar/data/109343...,https://www.sec.gov/Archives/edgar/data/109343...
9,1028087,https://www.sec.gov/Archives/edgar/data/102808...,https://www.sec.gov/Archives/edgar/data/102808...
